## Building the GBDT here

In [1]:
##Let us write spark code here for page rank
import os
import sys

spark_home = os.environ['SPARK_HOME'] = '/usr/local/Cellar/apache-spark/1.5.0/libexec/'
if not spark_home:
    raise ValueError('SPARK_HOME enviroment variable is not set')
sys.path.insert(0,os.path.join(spark_home,'python'))
sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.8.2.1-src.zip'))
execfile(os.path.join(spark_home,'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.5.0
      /_/

Using Python version 2.7.10 (default, May 28 2015 17:04:42)
SparkContext available as sc, HiveContext available as sqlContext.


Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


In [4]:
#%%writefile GBDT.py
import sys
from pyspark import SparkContext
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import SparseVector
from types import *

def parsePoint(point):
    ##exclude the first 13 features always as they are numerical
    return [(i,j)for i,j in enumerate(point.split(',')[14:])]


    
def createOneHotDict(inputData):
    sampleDistinctFeats = (inputData #.flatMap(filterNumbers)
                         .flatMap(lambda x: x)
                      .distinct())
    return (sampleDistinctFeats
                           .zipWithIndex()
                             .collectAsMap())    
# TODO: Replace <FILL IN> with appropriate code
def oneHotEncoding(rawFeats, OHEDict, numOHEFeats,first13):
    rawFeats=sorted(rawFeats)
    num=[]
    arr=[]
    for j,i in enumerate(first13):
        if i != '':
            num.append(j)
            arr.append(i)
    OHEFeats=[]
    for i in OHEDict.keys():
        if i in rawFeats:
            OHEFeats.append(i)
            
            
    return SparseVector(26+13,sorted(num+\
                            [OHEDict[i]+13 for i in OHEFeats]),\
                        arr+[1.0 for i in range(len(OHEFeats))]) #[i for i in first13]+

def parseOHEPoint(point, OHEDict, numOHEFeats):
  
    featurelist=point.strip().split(',')
    
    return LabeledPoint(featurelist[0],oneHotEncoding([(i,j) for i,j in enumerate(featurelist[14:]) ],\
                                                      OHEDict, 26+13,[i for i in featurelist[1:14]]))

def maaro(z):
    if z[1]=='':
        return (-1,0)
    else:
        return(z,1)
#print sampleOneOHEFeat
if __name__== '__main__':
    sys.argv[1]='dac_sample.txt'
    #sc = SparkContext(appName="GBDT")
    dataRDD=sc.textFile(sys.argv[1]).map(lambda x: x.replace('\t',','))
    #Now let us create labeled point from data
    dataRDDParsed=dataRDD.map(parsePoint).cache()
    featSet=dataRDDParsed.flatMap(lambda x: x).map(maaro).reduceByKey(lambda a,b: a+b).takeOrdered(26,lambda (k,v): -v)
    #reduceByKey(lambda x,y:x+y).takeOrdered(25,lambda (k,v):-v)
    #print featSet
    #OHEdict=createOneHotDict(dataRDDParsed,featSet)
    OHEdict={}
    for i,x in enumerate(featSet):
#         print i,x
        OHEdict[x[0]]=i
   
    #print oneHotEncoding(dataRDDParsed,OHEdict,numSampleOHEFeats,)
    #Now let us create a dictionary of points
#     weights=[.8,.1,.1]
#     seed=42
#     trainRDD,validateRDD,testRDD=dataRDD.randomSplit(weights,seed)
#     OHETrainData = trainRDD.map(lambda point: parseOHEPoint(point, OHEdict, 39))
    OHETrainData = dataRDD.map(lambda point: parseOHEPoint(point, OHEdict, 39))
#     print OHETrainData.take(1)
#     print OHETrainData.count()

In [5]:
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils

# Load and parse the data file.
trainingData = OHETrainData
# Split the data into training and test sets (30% held out for testing)
#(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a GradientBoostedTrees model.
#  Notes: (a) Empty categoricalFeaturesInfo indicates all features are continuous.
#         (b) Use more iterations in practice.
model = (GradientBoostedTrees.trainClassifier(trainingData, loss = 'logLoss', numIterations=2, 
    categoricalFeaturesInfo={}, learningRate = 0.1, maxDepth = 7, maxBins = 2))

#     categoricalFeaturesInfo={13:2, 14:2, 15:2, 16:2, 17:2, 18:2, 19:2, 20:2, 21:2, 22:2, 23:2, 24:2, 25:2, 26:2, 
#     27:2, 28:2, 29:2, 30:2, 31:2, }, 


# Evaluate model on test instances and compute test error
#predictions = model.predict(testData.map(lambda x: x.features))
#labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
#testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(testData.count())
#print('Test Error = ' + str(testErr))
#print('Learned classification GBT model:')


In [6]:
# print(model.toDebugString())
model.txt = model.toDebugString().split('\n')

all_lines = []
for line in model.txt:
    text = line.split()
    if text == []:
        pass
    else:
        all_lines.append(text)
        
starting_indent = 1
if_indents = {0: starting_indent}
if_num = 0

for i in range(len(all_lines)):
    if all_lines[i][0] == 'Tree':
        indents = starting_indent - 1
        bin_num = 0
        tabs = '\t'*starting_indent
        tree_num = int(all_lines[i][1].strip(':'))
        print tabs + '# Tree ' + str(tree_num)
        
    elif all_lines[i][0] == 'If':
        indents = indents + 1
        if_indents[if_num] = indents
        tabs = '\t'*indents
        print tabs + 'if ' + all_lines[i][1] + '[' + all_lines[i][2] + ']' + all_lines[i][3] + all_lines[i][4] + ':'
        if_num += 1
        
    elif all_lines[i][0] == 'Else':
        indents = if_indents[if_num - 1]
        tabs = '\t'*indents
        print tabs + 'else:'
        if_num -= 1
        
    elif all_lines[i][0] == 'Predict:':
        indents = indents + 1
        tabs = '\t'*indents
        print tabs + 'bin_num[' + str(tree_num) + '] = ' + str(bin_num)
        bin_num += 1
        indents = indents - 1



	# Tree 0
	if (feature[0]<=0.0):
		if (feature[10]<=1.0):
			if (feature[12]<=3.0):
				if (feature[19]<=0.0):
					if (feature[18]<=0.0):
						if (feature[13]<=0.0):
							if (feature[35]<=0.0):
								bin_num[0] = 0
							else:
								bin_num[0] = 1
						else:
							if (feature[8]<=35.0):
								bin_num[0] = 2
							else:
								bin_num[0] = 3
					else:
						if (feature[35]<=0.0):
							if (feature[20]<=0.0):
								bin_num[0] = 4
							else:
								bin_num[0] = 5
						else:
							if (feature[17]<=0.0):
								bin_num[0] = 6
							else:
								bin_num[0] = 7
				else:
					if (feature[8]<=35.0):
						if (feature[2]<=4.0):
							if (feature[7]<=7.0):
								bin_num[0] = 8
							else:
								bin_num[0] = 9
						else:
							if (feature[13]<=0.0):
								bin_num[0] = 10
							else:
								bin_num[0] = 11
					else:
						if (feature[35]<=0.0):
							if (feature[5]<=15.0):
								bin_num[0] = 12
							else:
								bin_num[0] = 13
						else:
							if (f

In [15]:
def get_bins(feature):
    bin_num = {} 
	# Tree 0
    if (feature[0]<=0.0):
		if (feature[10]<=1.0):
			if (feature[12]<=3.0):
				if (feature[19]<=0.0):
					if (feature[18]<=0.0):
						if (feature[13]<=0.0):
							if (feature[35]<=0.0):
								bin_num[0] = 0
							else:
								bin_num[0] = 1
						else:
							if (feature[8]<=35.0):
								bin_num[0] = 2
							else:
								bin_num[0] = 3
					else:
						if (feature[35]<=0.0):
							if (feature[20]<=0.0):
								bin_num[0] = 4
							else:
								bin_num[0] = 5
						else:
							if (feature[17]<=0.0):
								bin_num[0] = 6
							else:
								bin_num[0] = 7
				else:
					if (feature[8]<=35.0):
						if (feature[2]<=4.0):
							if (feature[7]<=7.0):
								bin_num[0] = 8
							else:
								bin_num[0] = 9
						else:
							if (feature[13]<=0.0):
								bin_num[0] = 10
							else:
								bin_num[0] = 11
					else:
						if (feature[35]<=0.0):
							if (feature[5]<=15.0):
								bin_num[0] = 12
							else:
								bin_num[0] = 13
						else:
							if (feature[6]<=2.0):
								bin_num[0] = 14
							else:
								bin_num[0] = 15
			else:
				if (feature[6]<=2.0):
					if (feature[35]<=0.0):
						if (feature[23]<=0.0):
							if (feature[2]<=4.0):
								bin_num[0] = 16
							else:
								bin_num[0] = 17
						else:
							if (feature[19]<=0.0):
								bin_num[0] = 18
							else:
								bin_num[0] = 19
					else:
						if (feature[18]<=0.0):
							if (feature[11]<=0.0):
								bin_num[0] = 20
							else:
								bin_num[0] = 21
						else:
							if (feature[17]<=0.0):
								bin_num[0] = 22
							else:
								bin_num[0] = 23
				else:
					if (feature[5]<=15.0):
						if (feature[18]<=0.0):
							if (feature[36]<=0.0):
								bin_num[0] = 24
							else:
								bin_num[0] = 25
						else:
							if (feature[3]<=2.0):
								bin_num[0] = 26
							else:
								bin_num[0] = 27
					else:
						if (feature[7]<=7.0):
							if (feature[8]<=35.0):
								bin_num[0] = 28
							else:
								bin_num[0] = 29
						else:
							if (feature[18]<=0.0):
								bin_num[0] = 30
							else:
								bin_num[0] = 31
		else:
			if (feature[5]<=15.0):
				if (feature[12]<=3.0):
					if (feature[20]<=0.0):
						if (feature[28]<=0.0):
							if (feature[19]<=0.0):
								bin_num[0] = 32
							else:
								bin_num[0] = 33
						else:
							if (feature[16]<=0.0):
								bin_num[0] = 34
							else:
								bin_num[0] = 35
					else:
						if (feature[35]<=0.0):
							if (feature[8]<=35.0):
								bin_num[0] = 36
							else:
								bin_num[0] = 37
						else:
							if (feature[17]<=0.0):
								bin_num[0] = 38
							else:
								bin_num[0] = 39
				else:
					if (feature[6]<=2.0):
						if (feature[27]<=0.0):
							if (feature[1]<=3.0):
								bin_num[0] = 40
							else:
								bin_num[0] = 41
						else:
							bin_num[0] = 42
					else:
						if (feature[27]<=0.0):
							if (feature[7]<=7.0):
								bin_num[0] = 43
							else:
								bin_num[0] = 44
						else:
							if (feature[25]<=0.0):
								bin_num[0] = 45
							else:
								bin_num[0] = 46
			else:
				if (feature[7]<=7.0):
					if (feature[2]<=4.0):
						if (feature[21]<=0.0):
							if (feature[25]<=0.0):
								bin_num[0] = 47
							else:
								bin_num[0] = 48
						else:
							if (feature[29]<=0.0):
								bin_num[0] = 49
							else:
								bin_num[0] = 50
					else:
						if (feature[20]<=0.0):
							if (feature[28]<=0.0):
								bin_num[0] = 51
							else:
								bin_num[0] = 52
						else:
							if (feature[12]<=3.0):
								bin_num[0] = 53
							else:
								bin_num[0] = 54
				else:
					if (feature[12]<=3.0):
						if (feature[19]<=0.0):
							if (feature[22]<=0.0):
								bin_num[0] = 55
							else:
								bin_num[0] = 56
						else:
							if (feature[21]<=0.0):
								bin_num[0] = 57
							else:
								bin_num[0] = 58
					else:
						if (feature[22]<=0.0):
							if (feature[29]<=0.0):
								bin_num[0] = 59
							else:
								bin_num[0] = 60
						else:
							if (feature[2]<=4.0):
								bin_num[0] = 61
							else:
								bin_num[0] = 62
    else:
		if (feature[5]<=15.0):
			if (feature[12]<=3.0):
				if (feature[6]<=2.0):
					if (feature[24]<=0.0):
						if (feature[18]<=0.0):
							if (feature[27]<=0.0):
								bin_num[0] = 63
							else:
								bin_num[0] = 64
						else:
							if (feature[36]<=0.0):
								bin_num[0] = 65
							else:
								bin_num[0] = 66
					else:
						if (feature[38]<=0.0):
							if (feature[9]<=0.0):
								bin_num[0] = 67
							else:
								bin_num[0] = 68
						else:
							if (feature[11]<=0.0):
								bin_num[0] = 69
							else:
								bin_num[0] = 70
				else:
					if (feature[3]<=2.0):
						if (feature[10]<=1.0):
							if (feature[28]<=0.0):
								bin_num[0] = 71
							else:
								bin_num[0] = 72
						else:
							if (feature[28]<=0.0):
								bin_num[0] = 73
							else:
								bin_num[0] = 74
					else:
						if (feature[28]<=0.0):
							if (feature[21]<=0.0):
								bin_num[0] = 75
							else:
								bin_num[0] = 76
						else:
							if (feature[23]<=0.0):
								bin_num[0] = 77
							else:
								bin_num[0] = 78
			else:
				if (feature[10]<=1.0):
					if (feature[6]<=2.0):
						if (feature[20]<=0.0):
							if (feature[22]<=0.0):
								bin_num[0] = 79
							else:
								bin_num[0] = 80
						else:
							if (feature[2]<=4.0):
								bin_num[0] = 81
							else:
								bin_num[0] = 82
					else:
						if (feature[25]<=0.0):
							if (feature[37]<=0.0):
								bin_num[0] = 83
							else:
								bin_num[0] = 84
						else:
							if (feature[21]<=0.0):
								bin_num[0] = 85
							else:
								bin_num[0] = 86
				else:
					if (feature[24]<=0.0):
						if (feature[27]<=0.0):
							if (feature[20]<=0.0):
								bin_num[0] = 87
							else:
								bin_num[0] = 88
						else:
							if (feature[8]<=35.0):
								bin_num[0] = 89
							else:
								bin_num[0] = 90
					else:
						if (feature[34]<=0.0):
							if (feature[1]<=3.0):
								bin_num[0] = 91
							else:
								bin_num[0] = 92
						else:
							if (feature[20]<=0.0):
								bin_num[0] = 93
							else:
								bin_num[0] = 94
		else:
			if (feature[10]<=1.0):
				if (feature[2]<=4.0):
					if (feature[19]<=0.0):
						if (feature[21]<=0.0):
							if (feature[7]<=7.0):
								bin_num[0] = 95
							else:
								bin_num[0] = 96
						else:
							if (feature[38]<=0.0):
								bin_num[0] = 97
							else:
								bin_num[0] = 98
					else:
						if (feature[22]<=0.0):
							if (feature[12]<=3.0):
								bin_num[0] = 99
							else:
								bin_num[0] = 100
						else:
							if (feature[32]<=0.0):
								bin_num[0] = 101
							else:
								bin_num[0] = 102
				else:
					if (feature[28]<=0.0):
						if (feature[19]<=0.0):
							if (feature[22]<=0.0):
								bin_num[0] = 103
							else:
								bin_num[0] = 104
						else:
							if (feature[12]<=3.0):
								bin_num[0] = 105
							else:
								bin_num[0] = 106
					else:
						if (feature[12]<=3.0):
							if (feature[6]<=2.0):
								bin_num[0] = 107
							else:
								bin_num[0] = 108
						else:
							if (feature[20]<=0.0):
								bin_num[0] = 109
							else:
								bin_num[0] = 110
			else:
				if (feature[2]<=4.0):
					if (feature[21]<=0.0):
						if (feature[29]<=0.0):
							if (feature[35]<=0.0):
								bin_num[0] = 111
							else:
								bin_num[0] = 112
						else:
							if (feature[9]<=0.0):
								bin_num[0] = 113
							else:
								bin_num[0] = 114
					else:
						if (feature[12]<=3.0):
							if (feature[22]<=0.0):
								bin_num[0] = 115
							else:
								bin_num[0] = 116
						else:
							if (feature[38]<=0.0):
								bin_num[0] = 117
							else:
								bin_num[0] = 118
				else:
					if (feature[12]<=3.0):
						if (feature[35]<=0.0):
							if (feature[25]<=0.0):
								bin_num[0] = 119
							else:
								bin_num[0] = 120
						else:
							if (feature[23]<=0.0):
								bin_num[0] = 121
							else:
								bin_num[0] = 122
					else:
						if (feature[24]<=0.0):
							if (feature[21]<=0.0):
								bin_num[0] = 123
							else:
								bin_num[0] = 124
						else:
							if (feature[31]<=0.0):
								bin_num[0] = 125
							else:
								bin_num[0] = 126
	# Tree 1
    if (feature[6]<=2.0):
		if (feature[7]<=7.0):
			if (feature[13]<=0.0):
				if (feature[35]<=0.0):
					if (feature[21]<=0.0):
						if (feature[0]<=0.0):
							if (feature[8]<=35.0):
								bin_num[1] = 0
							else:
								bin_num[1] = 1
						else:
							if (feature[5]<=15.0):
								bin_num[1] = 2
							else:
								bin_num[1] = 3
					else:
						if (feature[5]<=15.0):
							if (feature[17]<=0.0):
								bin_num[1] = 4
							else:
								bin_num[1] = 5
						else:
							if (feature[2]<=4.0):
								bin_num[1] = 6
							else:
								bin_num[1] = 7
				else:
					if (feature[2]<=4.0):
						if (feature[34]<=0.0):
							if (feature[0]<=0.0):
								bin_num[1] = 8
							else:
								bin_num[1] = 9
						else:
							if (feature[3]<=2.0):
								bin_num[1] = 10
							else:
								bin_num[1] = 11
					else:
						if (feature[37]<=0.0):
							if (feature[26]<=0.0):
								bin_num[1] = 12
							else:
								bin_num[1] = 13
						else:
							if (feature[31]<=0.0):
								bin_num[1] = 14
							else:
								bin_num[1] = 15
			else:
				if (feature[23]<=0.0):
					if (feature[8]<=35.0):
						if (feature[2]<=4.0):
							if (feature[21]<=0.0):
								bin_num[1] = 16
							else:
								bin_num[1] = 17
						else:
							if (feature[4]<=1897.0):
								bin_num[1] = 18
							else:
								bin_num[1] = 19
					else:
						if (feature[19]<=0.0):
							if (feature[11]<=0.0):
								bin_num[1] = 20
							else:
								bin_num[1] = 21
						else:
							if (feature[33]<=0.0):
								bin_num[1] = 22
							else:
								bin_num[1] = 23
				else:
					if (feature[25]<=0.0):
						if (feature[18]<=0.0):
							if (feature[28]<=0.0):
								bin_num[1] = 24
							else:
								bin_num[1] = 25
						else:
							if (feature[2]<=4.0):
								bin_num[1] = 26
							else:
								bin_num[1] = 27
					else:
						if (feature[35]<=0.0):
							if (feature[21]<=0.0):
								bin_num[1] = 28
							else:
								bin_num[1] = 29
						else:
							if (feature[12]<=3.0):
								bin_num[1] = 30
							else:
								bin_num[1] = 31
		else:
			if (feature[4]<=1897.0):
				if (feature[19]<=0.0):
					if (feature[20]<=0.0):
						if (feature[32]<=0.0):
							if (feature[13]<=0.0):
								bin_num[1] = 32
							else:
								bin_num[1] = 33
						else:
							if (feature[36]<=0.0):
								bin_num[1] = 34
							else:
								bin_num[1] = 35
					else:
						if (feature[15]<=0.0):
							if (feature[35]<=0.0):
								bin_num[1] = 36
							else:
								bin_num[1] = 37
						else:
							if (feature[28]<=0.0):
								bin_num[1] = 38
							else:
								bin_num[1] = 39
				else:
					if (feature[2]<=4.0):
						if (feature[21]<=0.0):
							if (feature[23]<=0.0):
								bin_num[1] = 40
							else:
								bin_num[1] = 41
						else:
							if (feature[12]<=3.0):
								bin_num[1] = 42
							else:
								bin_num[1] = 43
					else:
						if (feature[35]<=0.0):
							if (feature[18]<=0.0):
								bin_num[1] = 44
							else:
								bin_num[1] = 45
						else:
							if (feature[20]<=0.0):
								bin_num[1] = 46
							else:
								bin_num[1] = 47
			else:
				if (feature[21]<=0.0):
					if (feature[5]<=15.0):
						if (feature[19]<=0.0):
							if (feature[12]<=3.0):
								bin_num[1] = 48
							else:
								bin_num[1] = 49
						else:
							if (feature[13]<=0.0):
								bin_num[1] = 50
							else:
								bin_num[1] = 51
					else:
						if (feature[8]<=35.0):
							if (feature[2]<=4.0):
								bin_num[1] = 52
							else:
								bin_num[1] = 53
						else:
							if (feature[36]<=0.0):
								bin_num[1] = 54
							else:
								bin_num[1] = 55
				else:
					if (feature[17]<=0.0):
						if (feature[26]<=0.0):
							if (feature[35]<=0.0):
								bin_num[1] = 56
							else:
								bin_num[1] = 57
						else:
							if (feature[19]<=0.0):
								bin_num[1] = 58
							else:
								bin_num[1] = 59
					else:
						if (feature[2]<=4.0):
							if (feature[10]<=1.0):
								bin_num[1] = 60
							else:
								bin_num[1] = 61
						else:
							if (feature[30]<=0.0):
								bin_num[1] = 62
							else:
								bin_num[1] = 63
    else:
		if (feature[12]<=3.0):
			if (feature[25]<=0.0):
				if (feature[35]<=0.0):
					if (feature[29]<=0.0):
						if (feature[21]<=0.0):
							if (feature[24]<=0.0):
								bin_num[1] = 64
							else:
								bin_num[1] = 65
						else:
							if (feature[38]<=0.0):
								bin_num[1] = 66
							else:
								bin_num[1] = 67
					else:
						if (feature[20]<=0.0):
							if (feature[11]<=0.0):
								bin_num[1] = 68
							else:
								bin_num[1] = 69
						else:
							if (feature[1]<=3.0):
								bin_num[1] = 70
							else:
								bin_num[1] = 71
				else:
					if (feature[26]<=0.0):
						if (feature[5]<=15.0):
							if (feature[2]<=4.0):
								bin_num[1] = 72
							else:
								bin_num[1] = 73
						else:
							if (feature[11]<=0.0):
								bin_num[1] = 74
							else:
								bin_num[1] = 75
					else:
						if (feature[9]<=0.0):
							if (feature[34]<=0.0):
								bin_num[1] = 76
							else:
								bin_num[1] = 77
						else:
							if (feature[2]<=4.0):
								bin_num[1] = 78
							else:
								bin_num[1] = 79
			else:
				if (feature[3]<=2.0):
					if (feature[21]<=0.0):
						if (feature[2]<=4.0):
							if (feature[29]<=0.0):
								bin_num[1] = 80
							else:
								bin_num[1] = 81
						else:
							if (feature[11]<=0.0):
								bin_num[1] = 82
							else:
								bin_num[1] = 83
					else:
						if (feature[27]<=0.0):
							if (feature[15]<=0.0):
								bin_num[1] = 84
							else:
								bin_num[1] = 85
						else:
							if (feature[36]<=0.0):
								bin_num[1] = 86
							else:
								bin_num[1] = 87
				else:
					if (feature[17]<=0.0):
						if (feature[7]<=7.0):
							if (feature[5]<=15.0):
								bin_num[1] = 88
							else:
								bin_num[1] = 89
						else:
							if (feature[23]<=0.0):
								bin_num[1] = 90
							else:
								bin_num[1] = 91
					else:
						if (feature[8]<=35.0):
							if (feature[32]<=0.0):
								bin_num[1] = 92
							else:
								bin_num[1] = 93
						else:
							if (feature[7]<=7.0):
								bin_num[1] = 94
							else:
								bin_num[1] = 95
		else:
			if (feature[5]<=15.0):
				if (feature[35]<=0.0):
					if (feature[21]<=0.0):
						if (feature[8]<=35.0):
							if (feature[28]<=0.0):
								bin_num[1] = 96
							else:
								bin_num[1] = 97
						else:
							if (feature[34]<=0.0):
								bin_num[1] = 98
							else:
								bin_num[1] = 99
					else:
						if (feature[3]<=2.0):
							if (feature[18]<=0.0):
								bin_num[1] = 100
							else:
								bin_num[1] = 101
						else:
							if (feature[38]<=0.0):
								bin_num[1] = 102
							else:
								bin_num[1] = 103
				else:
					if (feature[21]<=0.0):
						if (feature[23]<=0.0):
							if (feature[26]<=0.0):
								bin_num[1] = 104
							else:
								bin_num[1] = 105
						else:
							if (feature[4]<=1897.0):
								bin_num[1] = 106
							else:
								bin_num[1] = 107
					else:
						if (feature[17]<=0.0):
							if (feature[19]<=0.0):
								bin_num[1] = 108
							else:
								bin_num[1] = 109
						else:
							if (feature[30]<=0.0):
								bin_num[1] = 110
							else:
								bin_num[1] = 111
			else:
				if (feature[18]<=0.0):
					if (feature[22]<=0.0):
						if (feature[4]<=1897.0):
							if (feature[27]<=0.0):
								bin_num[1] = 112
							else:
								bin_num[1] = 113
						else:
							if (feature[37]<=0.0):
								bin_num[1] = 114
							else:
								bin_num[1] = 115
					else:
						if (feature[21]<=0.0):
							if (feature[10]<=1.0):
								bin_num[1] = 116
							else:
								bin_num[1] = 117
						else:
							if (feature[14]<=0.0):
								bin_num[1] = 118
							else:
								bin_num[1] = 119
				else:
					if (feature[35]<=0.0):
						if (feature[17]<=0.0):
							if (feature[2]<=4.0):
								bin_num[1] = 120
							else:
								bin_num[1] = 121
						else:
							if (feature[4]<=1897.0):
								bin_num[1] = 122
							else:
								bin_num[1] = 123
					else:
						if (feature[10]<=1.0):
							if (feature[23]<=0.0):
								bin_num[1] = 124
							else:
								bin_num[1] = 125
						else:
							if (feature[8]<=35.0):
								bin_num[1] = 126
							else:
								bin_num[1] = 127
                                
    return bin_num
    
OHETrainDataDense = OHETrainData.map(lambda x: SparseVector.toArray(x.features))
OHETrainDataBins = OHETrainDataDense.map(lambda x: get_bins(x))
print OHETrainDataBins.take(5)



[{0: 74, 1: 82}, {0: 80, 1: 18}, {0: 111, 1: 112}, {0: 2, 1: 16}, {0: 71, 1: 64}]
